<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [28]:
import csv
from collections import namedtuple   # Convenient to store the data rows

DATA_FILE = './data/chipotle.tsv'

In [29]:
file_nested_list = []
with open(DATA_FILE, "r") as tsv_file:
    reader =  csv.reader(tsv_file, delimiter='\t')
    Order = namedtuple("Order", ["id", "qty", "item", "desc", "price"])
    
    for line in reader:
        order = Order(*line)
        file_nested_list.append(order)

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [30]:
header = file_nested_list[0]

In [31]:
data = file_nested_list[1:]

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [32]:
header

Order(id='order_id', qty='quantity', item='item_name', desc='choice_description', price='item_price')

In [59]:
data

[Order(id='1', qty='1', item='Chips and Fresh Tomato Salsa', desc='NULL', price='$2.39 '),
 Order(id='1', qty='1', item='Izze', desc='[Clementine]', price='$3.39 '),
 Order(id='1', qty='1', item='Nantucket Nectar', desc='[Apple]', price='$3.39 '),
 Order(id='1', qty='1', item='Chips and Tomatillo-Green Chili Salsa', desc='NULL', price='$2.39 '),
 Order(id='2', qty='2', item='Chicken Bowl', desc='[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]', price='$16.98 '),
 Order(id='3', qty='1', item='Chicken Bowl', desc='[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]', price='$10.98 '),
 Order(id='3', qty='1', item='Side of Chips', desc='NULL', price='$1.69 '),
 Order(id='4', qty='1', item='Steak Burrito', desc='[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]', price='$11.75 '),
 Order(id='4', qty='1', item='Steak Soft Tacos', desc='[Tomatillo Green Chili Salsa, [Pinto Bean

To calculate the average price of an order:

1. [Total order] Find out how many orders there are - no. of unique order IDs
1. [Total price] Sum the price across all orders.
1. Divide `Total price` by `Total order`.

In [34]:
# Get all the order IDs and put them in a set to get a unique collection of IDs.
total_order = len({order.id for order in data})
total_order

1834

In [35]:
# Get all the prices and strip any leading/trailing spaces.
price_list = [order.price.strip() for order in data]
# print(len(price_list))

# Remove `$` sign if present.
price_list_formatted = [price[1:] if "$" in price else price for price in price_list]
# print(len(price_list_formatted))

total_price = sum([float(price) for price in price_list_formatted])
total_price

34500.16000000046

In [36]:
avg_order_price = total_price / total_order
print("average price of an order: ${:.2f}".format(avg_order_price))

average price of an order: $18.81


### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [37]:
# Get all the `Canned Soda` and `Canned Soft Drink` orders.
sodas = [order.desc for order in data if order.item in ["Canned Soda", "Canned Soft Drink"]]
sodas

['[Sprite]',
 '[Dr. Pepper]',
 '[Mountain Dew]',
 '[Sprite]',
 '[Dr. Pepper]',
 '[Diet Dr. Pepper]',
 '[Coca Cola]',
 '[Diet Coke]',
 '[Diet Dr. Pepper]',
 '[Coca Cola]',
 '[Dr. Pepper]',
 '[Coca Cola]',
 '[Diet Coke]',
 '[Mountain Dew]',
 '[Mountain Dew]',
 '[Dr. Pepper]',
 '[Mountain Dew]',
 '[Diet Dr. Pepper]',
 '[Mountain Dew]',
 '[Coke]',
 '[Coca Cola]',
 '[Sprite]',
 '[Diet Coke]',
 '[Coke]',
 '[Coke]',
 '[Lemonade]',
 '[Sprite]',
 '[Diet Coke]',
 '[Coca Cola]',
 '[Diet Coke]',
 '[Diet Coke]',
 '[Mountain Dew]',
 '[Coke]',
 '[Coke]',
 '[Coke]',
 '[Sprite]',
 '[Coke]',
 '[Coke]',
 '[Dr. Pepper]',
 '[Coca Cola]',
 '[Dr. Pepper]',
 '[Coke]',
 '[Diet Dr. Pepper]',
 '[Diet Coke]',
 '[Lemonade]',
 '[Coke]',
 '[Diet Coke]',
 '[Diet Coke]',
 '[Diet Coke]',
 '[Diet Coke]',
 '[Nestea]',
 '[Diet Coke]',
 '[Dr. Pepper]',
 '[Coke]',
 '[Sprite]',
 '[Coke]',
 '[Lemonade]',
 '[Coke]',
 '[Coke]',
 '[Diet Coke]',
 '[Diet Coke]',
 '[Coca Cola]',
 '[Coca Cola]',
 '[Lemonade]',
 '[Lemonade]',
 '[Diet

In [38]:
# Find the unique sodas by using `set` and remove the square brackets from each soda.
unique_sodas = [soda.strip('[]') for soda in set(sodas)]
unique_sodas

['Diet Coke',
 'Mountain Dew',
 'Coke',
 'Diet Dr. Pepper',
 'Sprite',
 'Dr. Pepper',
 'Lemonade',
 'Nestea',
 'Coca Cola']

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [39]:
# Assume sauces are considered a topping.
# Get the description(toppings) of all the burrito orders.
burritos = [order.desc for order in data if "burrito" in order.item.lower()]
burritos

['[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]',
 '[Fresh Tomato Salsa, [Rice, Black Beans, Pinto Beans, Cheese, Sour Cream, Lettuce]]',
 '[Tomatillo-Green Chili Salsa (Medium), [Pinto Beans, Cheese, Sour Cream]]',
 '[Fresh Tomato Salsa (Mild), [Black Beans, Rice, Cheese, Sour Cream, Lettuce]]',
 '[[Fresh Tomato Salsa (Mild), Tomatillo-Green Chili Salsa (Medium), Tomatillo-Red Chili Salsa (Hot)], [Rice, Cheese, Sour Cream, Lettuce]]',
 '[[Tomatillo-Green Chili Salsa (Medium), Tomatillo-Red Chili Salsa (Hot)], [Pinto Beans, Rice, Cheese, Sour Cream, Guacamole, Lettuce]]',
 '[[Tomatillo-Green Chili Salsa (Medium), Roasted Chili Corn Salsa (Medium)], [Black Beans, Rice, Sour Cream, Lettuce]]',
 '[Tomatillo-Green Chili Salsa (Medium), [Pinto Beans, Rice, Cheese, Sour Cream]]',
 '[[Roasted Chili Corn Salsa (Medium), Fresh Tomato Salsa (Mild)], [Rice, Black Beans, Sour Cream]]',
 '[Fresh Tomato Salsa, [Rice, Pinto Beans, C

In [40]:
no_of_burrito_order = len(burritos)
no_of_burrito_order

1172

In [41]:
# Flatten the toppings of each burrito
flattened_burritos = [burrito.replace('[', '').replace(']', '') for burrito in burritos]
# print(flattened_burritos)

# Calculate the number of toppings per burrito.
burrito_toppings = [len(burrito.split(',')) for burrito in flattened_burritos]
# print(burrito_toppings)

total_burrito_toppings = sum(burrito_toppings)
total_burrito_toppings

6323

In [42]:
avg_topping_per_burrito = total_burrito_toppings / no_of_burrito_order
print("average topping per burrito: {:.0f}".format(avg_topping_per_burrito))

average topping per burrito: 5


### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [51]:
# Get all chip orders.
chips = [order for order in data if "chips" in order.item.lower()]
chips

[Order(id='1', qty='1', item='Chips and Fresh Tomato Salsa', desc='NULL', price='$2.39 '),
 Order(id='1', qty='1', item='Chips and Tomatillo-Green Chili Salsa', desc='NULL', price='$2.39 '),
 Order(id='3', qty='1', item='Side of Chips', desc='NULL', price='$1.69 '),
 Order(id='5', qty='1', item='Chips and Guacamole', desc='NULL', price='$4.45 '),
 Order(id='7', qty='1', item='Chips and Guacamole', desc='NULL', price='$4.45 '),
 Order(id='8', qty='1', item='Chips and Tomatillo-Green Chili Salsa', desc='NULL', price='$2.39 '),
 Order(id='10', qty='1', item='Chips and Guacamole', desc='NULL', price='$4.45 '),
 Order(id='13', qty='1', item='Chips and Fresh Tomato Salsa', desc='NULL', price='$2.39 '),
 Order(id='15', qty='1', item='Chips and Tomatillo-Green Chili Salsa', desc='NULL', price='$2.39 '),
 Order(id='16', qty='1', item='Side of Chips', desc='NULL', price='$1.69 '),
 Order(id='18', qty='1', item='Chips and Guacamole', desc='NULL', price='$4.45 '),
 Order(id='18', qty='1', item='Ch

In [54]:
from collections import defaultdict

chip_orders = defaultdict(lambda: 'Not Found')

for chip in chips:
    
    chip_formatted = chip.item.replace('-', ' ')
    
    if chip_orders[chip_formatted] != 'Not Found':
        chip_orders[chip_formatted] += int(chip.qty)
    
    
    else:
        chip_orders[chip_formatted] = int(chip.qty)

chip_orders

defaultdict(<function __main__.<lambda>()>,
            {'Chips and Fresh Tomato Salsa': 130,
             'Chips and Tomatillo Green Chili Salsa': 78,
             'Side of Chips': 110,
             'Chips and Guacamole': 506,
             'Chips': 230,
             'Chips and Tomatillo Red Chili Salsa': 75,
             'Chips and Roasted Chili Corn Salsa': 41,
             'Chips and Mild Fresh Tomato Salsa': 1})

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!


### Which taco toppings are demanded more?

In [45]:
tacos = [order.desc for order in data if "tacos" in order.item.lower()]
# tacos

In [46]:
# Flatten the toppings of each taco.
flattened_tacos = [taco.replace('[', '').replace(']', '') for taco in tacos]
# flattened_tacos

# Split the toppings of each taco into separate elements.
taco_toppings = [taco.split(',') for taco in flattened_tacos]
# taco_toppings

In [47]:
taco_toppings_record = defaultdict(lambda: 'Not Found')

for taco in taco_toppings:
    
    for topping in taco:
        
        topping = topping.strip()
    
        if taco_toppings_record[topping] != 'Not Found':
            taco_toppings_record[topping] += 1


        else:
            taco_toppings_record[topping] = 1
            
taco_toppings_record

defaultdict(<function __main__.<lambda>()>,
            {'Tomatillo Green Chili Salsa': 20,
             'Pinto Beans': 40,
             'Cheese': 293,
             'Sour Cream': 201,
             'Lettuce': 204,
             'Roasted Chili Corn Salsa': 75,
             'Fajita Vegetables': 63,
             'Rice': 159,
             'Black Beans': 70,
             'Fresh Tomato Salsa': 105,
             'Tomatillo-Red Chili Salsa (Hot)': 30,
             'Tomatillo Red Chili Salsa': 54,
             'Guacamole': 80,
             'Roasted Chili Corn Salsa (Medium)': 18,
             'Fajita Veggies': 19,
             'Fresh Tomato Salsa (Mild)': 41,
             'Tomatillo-Green Chili Salsa (Medium)': 15,
             'Adobo-Marinated and Grilled Steak': 1,
             'Adobo-Marinated and Grilled Chicken': 1,
             'Fresh Tomato (Mild)': 1})